In [1]:
%load_ext autoreload
%autoreload 2

# Simple example CBPS using Lalonde

In [2]:
import sys
sys.path.append('..')
from src.cbps_torch import CBPS
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [4]:
# Read lalonde data
df = pd.read_csv("../data/lalonde_psid.csv")
df.head()

,age,education,black,hispanic,married,nodegree,re74,re75,re78,u74,u75,treat
0,37,11,1,0,1,1,0.0,0.0,9930.05,1,1,1
1,22,9,0,1,0,1,0.0,0.0,3595.89,1,1,1
2,30,12,1,0,0,0,0.0,0.0,24909.50,1,1,1
3,27,11,1,0,0,1,0.0,0.0,7506.15,1,1,1
4,33,8,1,0,0,1,0.0,0.0,289.79,1,1,1


In [5]:
w, y = df.treat.values, df.re78.values
X = df.drop(columns=["treat", "re78"]).values
X = MinMaxScaler().fit_transform(X)

In [6]:
print(f" Our effect benchmark is given by: {y[w == 1].mean() - y[w == 0].mean()}")

 Our effect benchmark is given by: -15204.775555988717


In [9]:
cbps = CBPS(X = X,
        W = w,
        estimand="ATT",
        intercept=True,
        noi=False,
        niter=10000,
        lr=0.001,
        reg=0.0001,
)
weights = cbps.weights()

Optimizing CBPS...: 100%|██████████| 10000/10000 [00:02<00:00, 3599.02it/s]


In [10]:
y[w == 1].mean() - np.average(y[w == 0], weights=weights.data.cpu().numpy())

1932.8206145321874